In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

%matplotlib tk

Using TensorFlow backend.
/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


ImportError: cannot import name 'abs'

In [10]:
###データ読み込み
###学習データ
dataframe = pandas.read_csv('2lstm_testdata_kubo_up.csv', usecols=[0,1], engine='python')
###テストデータ
dataframe2 = pandas.read_csv('2lstm_testdata_kubo_70.csv', usecols=[0,1], engine='python')

#plt.figure(figsize=(80,10))
#plt.plot(dataframe2)
#plt.show()
#print(dataframe.head())

In [11]:
###np.arrayに変換し、キャスト
dataset = dataframe.values
dataset2 = dataframe2.values
dataset = dataset.astype('float32')
dataset2 = dataset2.astype('float32')
###データを標準化
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
dataset2 = scaler.fit_transform(dataset2)

#print(dataset)
#plt.figure(figsize=(80,10))
#plt.plot(dataset)
#plt.show()

###学習データ切り分け
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(dataset.shape[1])

###学習とテストで切り分ける場合
#print(len(dataset))
#print(len(dataset2))
#train_size = len(dataset)
#test_size = len(dataset2)
#train, test = dataset[:,:], dataset2[:,:]

2


In [12]:
###データを入力データと出力データにわける関数
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        xset = []
        for j in range(dataset.shape[1]):
            a = dataset[i:(i+look_back), j]
            xset.append(a)
        dataY.append(dataset[i + look_back, 1])      
        dataX.append(xset)
    return numpy.array(dataX), numpy.array(dataY)

### X=t, Y=t+1に切り分け
look_back = 300
trainX, trainY = create_dataset(train, look_back)#学習データ
testX, testY = create_dataset(test, look_back)#教師データ

print(trainX.shape[0])
print(trainX.shape[1])
print(trainX.shape[2])
print(trainX)
### 3次元配列に変換[samples, time steps(number of variables), features] 
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))
print(trainX)

1482
2
300
[[[0.9035064  0.9353831  0.9629561  ... 0.7220943  0.686618   0.651718  ]
  [0.75275004 0.75998455 0.77165246 ... 0.8883687  0.91346806 0.93419963]]

 [[0.9353831  0.9629561  0.9919632  ... 0.686618   0.651718   0.6200747 ]
  [0.75998455 0.77165246 0.78698814 ... 0.91346806 0.93419963 0.9407447 ]]

 [[0.9629561  0.9919632  0.978621   ... 0.651718   0.6200747  0.5785614 ]
  [0.77165246 0.78698814 0.8006517  ... 0.93419963 0.9407447  0.935231  ]]

 ...

 [[0.96123755 0.9932614  0.95272654 ... 0.01586908 0.02081843 0.04710713]
  [0.70468944 0.6879443  0.7025147  ... 0.05249304 0.04506275 0.04068199]]

 [[0.9932614  0.95272654 0.91661084 ... 0.02081843 0.04710713 0.07276058]
  [0.6879443  0.7025147  0.7250038  ... 0.04506275 0.04068199 0.03301157]]

 [[0.95272654 0.91661084 0.8821579  ... 0.04710713 0.07276058 0.09949633]
  [0.7025147  0.7250038  0.7356089  ... 0.04068199 0.03301157 0.01478451]]]
[[[0.9035064  0.9353831  0.9629561  ... 0.7220943  0.686618   0.651718  ]
  [0.7527

In [5]:
### LSTMネットワーク作成
model = Sequential()
model.add(LSTM(100, input_shape=(testX.shape[1], look_back)))	#input_shape：変数数、遡る時間数
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=20)#過学習防止
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1, callbacks=[early_stopping])

Epoch 1/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0172
Epoch 2/100
  83/1482 [>.............................] - ETA: 2s - loss: 0.0081

C:\Users\hai\Anaconda3\envs\textgen\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1482/1482 [==============================] - 3s 2ms/step - loss: 0.0051
Epoch 3/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0039
Epoch 4/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0033
Epoch 5/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0031
Epoch 6/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0021
Epoch 7/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0020
Epoch 8/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0020
Epoch 9/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0016
Epoch 10/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0017
Epoch 11/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0014
Epoch 12/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0013
Epoch 13/100
1482/1482 [==============================] - 3s 2ms/step - loss: 0.0014


1482/1482 [==============================] - 3s 2ms/step - loss: 4.0340e-04
Epoch 96/100
1482/1482 [==============================] - 3s 2ms/step - loss: 3.9353e-04
Epoch 97/100
1482/1482 [==============================] - 3s 2ms/step - loss: 3.9795e-04
Epoch 98/100
1482/1482 [==============================] - 3s 2ms/step - loss: 3.7651e-04
Epoch 99/100
1482/1482 [==============================] - 3s 2ms/step - loss: 4.2783e-04
Epoch 100/100
1482/1482 [==============================] - 3s 2ms/step - loss: 3.5448e-04


In [13]:
### 作成したモデルで予測
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

###予想データ、Yデータを標準化前の数字に戻す
#scalerが元のデータセットと同じ形でないと受け付けられないため存在していたカラムの数だけ値を0で埋める
print(dataset.shape[1])
pad_col = numpy.zeros(dataset.shape[1]-1)
def pad_array(val):
    return numpy.array([numpy.insert(pad_col, 0, x) for x in val])
    
trainPredict = scaler.inverse_transform(pad_array(trainPredict))
trainY = scaler.inverse_transform(pad_array(trainY))
testPredict = scaler.inverse_transform(pad_array(testPredict))
testY = scaler.inverse_transform(pad_array(testY))

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

#print(len(testPredict))

2
Train Score: 1.48 RMSE
Test Score: 4.29 RMSE


In [17]:
print(testY[:,0])
print(testPredict[:,0])
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset2)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
#testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset2)-1, :] = testPredict

# plot baseline and predictions

plt.plot(scaler.inverse_transform(dataset))
#plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
#plt.plot(testY)

plt.show()

[1.70357542e+01 2.02783929e+01 2.34360765e+01 2.62848705e+01
 2.93060036e+01 3.23933952e+01 3.54182575e+01 3.87202527e+01
 4.25339196e+01 4.64379329e+01 5.01405863e+01 5.39079674e+01
 5.76069212e+01 6.15867638e+01 6.58132602e+01 7.03959252e+01
 7.43718591e+01 7.75132548e+01 8.03104066e+01 8.28737086e+01
 8.47808812e+01 8.63317350e+01 8.70421098e+01 8.64130039e+01
 8.41640967e+01 8.20862257e+01 7.98025525e+01 7.70241509e+01
 7.35172892e+01 7.00486196e+01 6.69140066e+01 6.43171665e+01
 6.25786763e+01 6.13594018e+01 5.99194677e+01 5.92986993e+01
 5.90470430e+01 5.93801505e+01 6.03526800e+01 6.10545618e+01
 6.28542999e+01 6.44547021e+01 6.77136983e+01 6.99985940e+01
 7.29973443e+01 7.59300264e+01 7.86913988e+01 8.09214325e+01
 8.28386799e+01 8.38539162e+01 8.38446671e+01 8.34048543e+01
 8.25524292e+01 8.12644969e+01 7.92637876e+01 7.59980623e+01
 7.15331050e+01 6.65748349e+01 6.11943387e+01 5.60884151e+01
 5.13278347e+01 4.67244410e+01 4.25266651e+01 3.81836433e+01
 3.38131529e+01 2.925196